In [1]:
import sys
sys.path.append("../..")

In [2]:
from src.marks_reader import *
from src.prediction_saver import *
from src.competition_data import *
from src.simulator import simulate_all_events
from src.ponderator import ponderate_all_events
from events_data import events, event_params
from datetime import datetime

In [3]:
entry_list = load_json('entry_list.json')

competition = CompetitionData(
    name="World Athletics Championships Oregon22", 
    events_data=events, 
    events_params=event_params,
    start_date=datetime(2022, 7, 15),
    entry_list=entry_list,
)

event_folder = 'classified'
only = ['atl_jab']
years_weight = {
    2022: 4,
    2021: 2,
    2020: 1,
}

In [4]:
data = read_marks(
    folder_path=event_folder, 
    only=only,
    limit_date=competition.start_date
)

print(data)

{'atl_jab': {'female': {'AUS_alexandra_roberts':          Date  Result
0  2022-01-15   51.58
1  2022-02-09   49.72
2  2022-02-19   48.52
3  2022-03-06   45.37
4  2022-03-19   54.43
5  2022-04-01   55.61
6  2022-04-09   51.03
7  2022-04-21   55.14
8  2022-05-28   52.32
9  2022-06-04   50.47
10 2022-06-07   56.34
11 2021-02-27   55.69
12 2021-03-25   57.68
13 2021-04-10   54.72
14 2021-04-16   56.32
15 2021-05-03   55.55
16 2021-05-14   52.51
17 2021-05-27   48.12
18 2021-07-18   48.72
19 2020-09-12   48.23
20 2020-10-04   50.84
21 2020-10-25   44.08
22 2020-11-14   51.04
23 2020-12-16   52.77, 'AUS_kathryn_mitchell':          Date  Result
0  2022-06-11   57.36
1  2022-06-18   55.11
2  2021-01-23   60.25
3  2021-04-10   63.49
4  2021-04-15   57.55
5  2021-04-17   63.34
6  2021-04-29   60.50
7  2021-06-05   61.67
8  2021-06-12   63.50
9  2021-06-19   57.66
10 2021-07-11   56.25
11 2021-08-03   61.85
12 2021-08-06   61.82
13 2020-01-27   59.82, 'AUS_kelsey-lee_barber':          Date  Resul

In [5]:
ponderated_data = ponderate_all_events(
    data=data, 
    competition=competition,
    years_weight=years_weight,   
)

In [6]:
prediction = simulate_all_events(
    data=ponderated_data, 
    competition=competition, 
    top=8, 
    times=1000, 
    models_folder='models',
    override_models=True,
    only=only,
    logs=True, 
)

Creating athletes models for event atl_jab and sex male...
(1.32 %) Creating kde model of AUS_cameron_mcentyre...
(2.63 %) Creating kde model of AUS_cruz_hogan...
(3.95 %) Creating kde model of BAH_keyshawn_strachan...
(5.26 %) Creating kde model of BEL_timothy_herman...
(6.58 %) Creating kde model of BIH_dejan_mileusnić...
(7.89 %) Creating kde model of BRA_luiz_mauricio_da_silva...
(9.21 %) Creating kde model of BRA_pedro_henrique_rodrigues...
(10.53 %) Creating kde model of COL_arley_ibarguen...
(11.84 %) Creating kde model of COL_billy_julio_lópez...
(13.16 %) Creating kde model of CZE_jakub_vadlejch...
(14.47 %) Creating kde model of CZE_martin_konečný...
(15.79 %) Creating kde model of CZE_vítězslav_veselý...
(17.11 %) Creating kde model of DEN_arthur_w...
(18.42 %) Creating kde model of EGY_ihab_abdelrahman...
(19.74 %) Creating kde model of ESP_manu_quijera...
(21.05 %) Creating kde model of FIN_jami_kinnunen...
(22.37 %) Creating kde model of FIN_jarmo_marttila...
(23.68 %) Cr

In [ ]:
save_prediction(
    results=prediction, 
    competition=competition, 
    top=8,
)